In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import geocoder

# Using BeautifulSoup scraping from wikipedia

In [2]:
url = "en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

r  = requests.get("http://" +url)

data = r.text

soup = BeautifulSoup(data,'lxml')

# Reading html and append table to df dataframe

In [3]:
df = []
for i in soup.find_all("table",class_="wikitable sortable")[0].find_all("tr"):
    ls = []
    for j in i.find_all("td"):
        ls.append(j.get_text())
    df.append(ls)
df = pd.DataFrame(df)

In [4]:
df.head()

,0,1,2
0,None,None,None
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


# Replace row with index = 0 and change columns name to correct name

In [5]:
df.drop(index=0,inplace=True)
df.columns=['PostalCode','Borough','Neighborhood']

In [6]:
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n
5,M5A,Downtown Toronto,Harbourfront\n


# Clean Data

In [7]:
# Remove EOL "\n" String in dataset
for i in df.columns:
    df[i] = df[i].str.replace("\n","",regex=True)

In [8]:
# Remove data with Borough "Not Assigned"
df = df[(df['Borough'] != "Not assigned")]

In [9]:
# If Borough has value and Neighborhood "Not assigned". the neighborhood will be the same as the borough.
temp = df['Neighborhood'] == "Not assigned"
df[temp]['Neighborhood'] = df[temp]['Borough']

In [10]:
# Merge all the cells with same 'PostalCode'

# Create a new dataframe and concentrate all string in same 'Postal Code' using pd.str.cat and append it to new data
df_concentrate = pd.DataFrame(data=None,columns = df.columns)
for i in df['PostalCode'].unique():
    df_concentrate = df_concentrate.append({'PostalCode':i,'Borough':df[df['PostalCode'] == i]['Borough'].iloc[0],'Neighborhood':df[df['PostalCode'] == i]['Neighborhood'].str.cat(sep = ", ")},ignore_index=True)    

In [13]:
df_concentrate['Latiude']=np.NaN
df_concentrate['Longtitude']=np.NaN

In [14]:
df_concentrate

,PostalCode,Borough,Neighborhood,Latiude,Longtitude
0,M3A,North York,Parkwoods,NaN,NaN
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,Harbourfront,NaN,NaN
3,M6A,North York,"Lawrence Heights, Lawrence Manor",NaN,NaN
4,M7A,Downtown Toronto,Queen's Park,NaN,NaN
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",NaN,NaN
99,M4Y,Downtown Toronto,Church and Wellesley,NaN,NaN
100,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,NaN,NaN
101,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So...",NaN,NaN


In [ ]:
for i in df_concentrate.index:
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(df_concentrate['PostalCode'].iloc[i]))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    df_concentrate['Latiude'] = latitude
    df_concentrate['Longtitude'] = longitude

In [35]:
lat_lng_coords = None

# loop until you get the coordinates
g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
lat_lng_coords = g.latlng
    

In [37]:
print(lat_lng_coords)

None
